In [15]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

import plotly.express as px


!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.53 s (started: 2021-01-26 18:53:08 +00:00)


In [16]:
data = pd.read_csv('/content/weatherHistory.csv')

time: 249 ms (started: 2021-01-26 18:53:10 +00:00)


In [17]:
data

Formatted Date  ...                           Daily Summary
0      2006-04-01 00:00:00.000 +0200  ...       Partly cloudy throughout the day.
1      2006-04-01 01:00:00.000 +0200  ...       Partly cloudy throughout the day.
2      2006-04-01 02:00:00.000 +0200  ...       Partly cloudy throughout the day.
3      2006-04-01 03:00:00.000 +0200  ...       Partly cloudy throughout the day.
4      2006-04-01 04:00:00.000 +0200  ...       Partly cloudy throughout the day.
...                              ...  ...                                     ...
96448  2016-09-09 19:00:00.000 +0200  ...  Partly cloudy starting in the morning.
96449  2016-09-09 20:00:00.000 +0200  ...  Partly cloudy starting in the morning.
96450  2016-09-09 21:00:00.000 +0200  ...  Partly cloudy starting in the morning.
96451  2016-09-09 22:00:00.000 +0200  ...  Partly cloudy starting in the morning.
96452  2016-09-09 23:00:00.000 +0200  ...  Partly cloudy starting in the morning.

[96453 rows x 12 columns]

time: 53 ms (started: 2021-01-26 18:53:10 +00:00)


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Formatted Date            96453 non-null  object 
 1   Summary                   96453 non-null  object 
 2   Precip Type               95936 non-null  object 
 3   Temperature (C)           96453 non-null  float64
 4   Apparent Temperature (C)  96453 non-null  float64
 5   Humidity                  96453 non-null  float64
 6   Wind Speed (km/h)         96453 non-null  float64
 7   Wind Bearing (degrees)    96453 non-null  float64
 8   Visibility (km)           96453 non-null  float64
 9   Loud Cover                96453 non-null  float64
 10  Pressure (millibars)      96453 non-null  float64
 11  Daily Summary             96453 non-null  object 
dtypes: float64(8), object(4)
memory usage: 8.8+ MB
time: 38.3 ms (started: 2021-01-26 18:53:11 +00:00)


In [19]:
#Preprocessing

time: 1.47 ms (started: 2021-01-26 18:53:11 +00:00)


In [20]:
data

Formatted Date  ...                           Daily Summary
0      2006-04-01 00:00:00.000 +0200  ...       Partly cloudy throughout the day.
1      2006-04-01 01:00:00.000 +0200  ...       Partly cloudy throughout the day.
2      2006-04-01 02:00:00.000 +0200  ...       Partly cloudy throughout the day.
3      2006-04-01 03:00:00.000 +0200  ...       Partly cloudy throughout the day.
4      2006-04-01 04:00:00.000 +0200  ...       Partly cloudy throughout the day.
...                              ...  ...                                     ...
96448  2016-09-09 19:00:00.000 +0200  ...  Partly cloudy starting in the morning.
96449  2016-09-09 20:00:00.000 +0200  ...  Partly cloudy starting in the morning.
96450  2016-09-09 21:00:00.000 +0200  ...  Partly cloudy starting in the morning.
96451  2016-09-09 22:00:00.000 +0200  ...  Partly cloudy starting in the morning.
96452  2016-09-09 23:00:00.000 +0200  ...  Partly cloudy starting in the morning.

[96453 rows x 12 columns]

time: 43.7 ms (started: 2021-01-26 18:53:11 +00:00)


In [21]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Drop Summary, Loud Cover, and Daily Summary columns
    df = df.drop(['Summary', 'Loud Cover', 'Daily Summary'], axis=1)
    
    # Fill missing values in Precip Type column
    df['Precip Type'] = df['Precip Type'].fillna(df['Precip Type'].mode()[0])
    
    # Extract date/time features from Formatted Date column
    df['Formatted Date'] = pd.to_datetime(data['Formatted Date'], format='%Y-%m-%d %H:%M:%S.%f %z')
    df['Year'] = df['Formatted Date'].apply(lambda x: x.year)
    df['Month'] = df['Formatted Date'].apply(lambda x: x.month)
    df['Day'] = df['Formatted Date'].apply(lambda x: x.day)
    df['Hour'] = df['Formatted Date'].apply(lambda x: x.hour)
    
    df = df.drop('Formatted Date', axis=1)
    
    # Binary encode Precip Type column
    df['Precip Type'] = df['Precip Type'].apply(lambda x: 1 if x == 'snow' else 0)
    
    # Split df into X and y
    y = df['Visibility (km)'].copy()
    X = df.drop('Visibility (km)', axis=1).copy()
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    
    return X_train, X_test, y_train, y_test

time: 30.1 ms (started: 2021-01-26 18:53:11 +00:00)


In [22]:
X_train, X_test, y_train, y_test = preprocess_inputs(data)

time: 2.93 s (started: 2021-01-26 18:53:11 +00:00)


In [23]:
y_train

13596     9.9820
79326    16.1000
72378    15.9229
13895    14.7154
37609    14.9569
          ...   
63206     6.9069
61404    11.2700
17730    11.2056
28030     6.3434
15725     5.8926
Name: Visibility (km), Length: 67517, dtype: float64

time: 5.89 ms (started: 2021-01-26 18:53:14 +00:00)


In [24]:
models = {
    "  Linear Regression": LinearRegression(),
    "      Decision Tree": DecisionTreeRegressor(),
    "K-Nearest Neighbors": KNeighborsRegressor()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

  Linear Regression trained.
      Decision Tree trained.
K-Nearest Neighbors trained.
time: 1.15 s (started: 2021-01-26 18:53:14 +00:00)


In [25]:
X_train

Precip Type  Temperature (C)  ...       Day      Hour
0        -0.355013         1.627626  ...  1.168975  0.073065
1        -0.355013         0.441592  ...  1.055362 -0.359778
2         2.816801        -1.472340  ... -1.671337  0.938750
3        -0.355013        -0.328167  ... -1.671337  1.660155
4         2.816801        -1.261296  ...  0.600913 -1.514026
...            ...              ...  ...       ...       ...
67512    -0.355013        -0.529909  ...  0.714525  0.361627
67513    -0.355013        -0.613047  ... -1.671337  0.073065
67514    -0.355013         0.033457  ...  0.032850  0.938750
67515     2.816801        -1.882801  ...  0.260075  1.515874
67516    -0.355013        -1.123507  ...  0.828137 -0.792621

[67517 rows x 11 columns]

time: 42.9 ms (started: 2021-01-26 18:53:15 +00:00)


In [26]:
for name, model in models.items():
    print(name + " R^2: {:.5f}".format(model.score(X_test, y_test)))

  Linear Regression R^2: 0.24017
      Decision Tree R^2: 0.50323
K-Nearest Neighbors R^2: 0.68630
time: 6.27 s (started: 2021-01-26 18:53:15 +00:00)


In [27]:
#Visualisation

time: 792 µs (started: 2021-01-26 18:53:21 +00:00)


In [28]:
log_acc = models['  Linear Regression'].score(X_test, y_test)
svm_acc = models['      Decision Tree'].score(X_test, y_test)
ann_acc = models['K-Nearest Neighbors'].score(X_test, y_test)
fig = px.bar(
    x=['Linear Regression', 'Decision Tree', 'K-Nearest Neighbors'],
    y=[log_acc, svm_acc, ann_acc],
    color=['Linear Regression', 'Decision Tree', 'K-Nearest Neighbors']
)

fig.show()

time: 6.3 s (started: 2021-01-26 18:53:21 +00:00)
